In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

### Pre processing

#### Đọc data vào từ folder

In [ ]:
def load_images_and_labels(df, root_path, target_size=(224,224)):
    images = []
    labels = []

    if not root_path.endswith('/'):
        root_path += '/'

    for idx, row in df.iterrows():
        image_path = os.path.join(root_path, row['crop_face_file_path'])

        print(f"Processing image {idx + 1}: {image_path}")

        # Check if image exists
        if not os.path.exists(image_path):
            print(f"Warning: Image {image_path} not found.")
            continue

        image = cv2.imread(image_path)

        # If image couldn't be loaded, skip it
        if image is None:
            print(f"Warning: Failed to load image {image_path}.")
            continue

        image = cv2.resize(image, target_size)  # Resize image to a standard size (224x224)

        images.append(image)

        # Convert multi-label to a list
        label = [row['natural'], row['sleepy_eye'], row['yawn'], row['rub_eye'], row['look_away']]
        labels.append(label)

    images = np.array(images, dtype=np.float32) # Chuyển sang float để chuẩn hóa sau
    labels = np.array(labels)
    return images, labels

In [ ]:
# Path to CSV file (adjust to your machine's path)
csv_path = r'/content/drive/MyDrive/ComputerVision/dataset_multi_label/training_data.csv'
df = pd.read_csv(csv_path)

In [ ]:
print(df.head())
print(f'Số mẫu: {df.shape[0]}')
print(f'Số cột: {df.shape[1]}')

               original_file_path                    crop_face_file_path  \
0  images/20250307_222829_001.jpg  cropped_faces/20250307_222829_001.jpg   
1  images/20250307_222829_002.jpg  cropped_faces/20250307_222829_002.jpg   
2  images/20250307_222829_003.jpg  cropped_faces/20250307_222829_003.jpg   
3  images/20250307_222829_004.jpg  cropped_faces/20250307_222829_004.jpg   
4  images/20250307_222829_005.jpg  cropped_faces/20250307_222829_005.jpg   

       labels  natural  sleepy_eye  yawn  rub_eye  look_away  
0     natural        1           0     0        0          0  
1     natural        1           0     0        0          0  
2     natural        1           0     0        0          0  
3  sleepy_eye        0           1     0        0          0  
4  sleepy_eye        0           1     0        0          0  
Số mẫu: 3436
Số cột: 8


#### Đọc image, labels vào từ folder (chỉ load lần đầu khi chưa save images, labels)

In [ ]:
root_path = '/content/drive/MyDrive/ComputerVision/dataset_multi_label/'
images, labels = load_images_and_labels(df, root_path)

Processing image 1: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_001.jpg
Processing image 2: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_002.jpg
Processing image 3: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_003.jpg
Processing image 4: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_004.jpg
Processing image 5: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_005.jpg
Processing image 6: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_006.jpg
Processing image 7: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_007.jpg
Processing image 8: /content/drive/MyDrive/ComputerVision/dataset_multi_label/cropped_faces/20250307_222829_008.jpg
Processing image 9: /content/drive/MyDrive/ComputerVision/dataset_multi_

In [ ]:
np.save('images.npy', images)
np.save('labels.npy', labels)

#### Save image and label

In [ ]:
images = np.load('images.npy')
labels = np.load('labels.npy')

#### Data augmentation

In [ ]:
# Data Augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescaling images to range [0, 1]
    rotation_range=15,  # Random rotation between -15 and 15 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill mode for empty pixels after transformation
)

# Data Augmentation for validation images (usually, we don't augment validation data)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

#### Apply the augmentation to the data

In [ ]:
# Lưu ý: ImageDataGenerator.flow() nhận đầu vào là dữ liệu numpy
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

### Build the CNN Model

In [ ]:
from tensorflow.keras.applications import MobileNetV2 # Hoặc EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(input_shape=(224, 224, 3), num_classes=5):
    # Load pre-trained base model (không bao gồm lớp fully connected cuối)
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Đóng băng các lớp của base model (để không huấn luyện lại chúng ban đầu)
    base_model.trainable = False # Quan trọng!

    x = base_model.output
    x = GlobalAveragePooling2D()(x) # Giảm chiều dữ liệu
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x) # Sử dụng sigmoid cho multi-label

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

model = build_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Train the Model CNN

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
# Optional: Giảm learning rate khi val_loss không cải thiện
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    'model_checkpoint.weights.h5',  # Changed the filepath to end with .weights.h5
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    verbose=1
)

epochs = 10
batch_size = 32

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[checkpoint_callback, early_stopping, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - accuracy: 0.3601 - loss: 0.5681
Epoch 1: val_loss improved from inf to 0.33893, saving model to model_checkpoint.weights.h5
84/84 ━━━━━━━━━━━━━━━━━━━━ 59s 555ms/step - accuracy: 0.3614 - loss: 0.5671 - val_accuracy: 0.6406 - val_loss: 0.3389 - learning_rate: 1.0000e-04
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.6060 - loss: 0.3849
Epoch 2: val_loss improved from 0.33893 to 0.28308, saving model to model_checkpoint.weights.h5
84/84 ━━━━━━━━━━━━━━━━━━━━ 31s 370ms/step - accuracy: 0.6063 - loss: 0.3847 - val_accuracy: 0.6797 - val_loss: 0.2831 - learning_rate: 1.0000e-04
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.6492 - loss: 0.3317
Epoch 3: val_loss improved from 0.28308 to 0.24828, saving model to model_checkpoint.weights.h5
84/84 ━━━━━━━━━━━━━━━━━━━━ 33s 390ms/step - accuracy: 0.6493 - loss: 0.3316 - val_accuracy: 0.7248 - val_loss: 0.2483 - learning_rate: 1.0000e-04
Epoch 4/10
84/84 ━━━

### Evaluate the Model

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.2548 - loss: 0.5848
Validation Loss: 0.6001015901565552
Validation Accuracy: 0.2345864623785019


###Save and Load the Model

In [ ]:
model.save('sleepiness_detection_model.h5')

### Test model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('sleepiness_detection_model.h5')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,599 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 2 (12.00 B)

### Predict

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def predict(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize image to 224x224
    image = image / 255.0  # Normalize the image

    image = np.expand_dims(image, axis=0)  # Add batch dimension

    prediction = model.predict(image)

    # Define the class labels
    class_labels = ['natural', 'sleepy_eye', 'yawn', 'rub_eye', 'look_away']

    # Print confidence for each class label
    for i, label in enumerate(class_labels):
        confidence = prediction[0][i]  # Get the confidence for the current label
        print(f"{label}: {confidence:.4f}")

    # Get the index of the class with the highest probability
    predicted_class_idx = np.argmax(prediction)

    # Get the predicted class label and its confidence
    predicted_class = class_labels[predicted_class_idx]
    confidence = prediction[0][predicted_class_idx]

    return predicted_class, confidence

# Example usage
image_path = '/content/drive/MyDrive/ComputerVision/dataset_multi_class/indian_2/natural/1000.jpg'
predicted_class, confidence = predict(image_path)
print(f"Predicted class with highest confidence: {predicted_class}, Confidence: {confidence:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
natural: 0.0311
sleepy_eye: 0.3771
yawn: 0.0140
rub_eye: 0.3256
look_away: 0.4021
Predicted class with highest confidence: look_away, Confidence: 0.4021
